In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! ls ../../models/

cached					  ERICA--exported.tar.gz
distilroberta-finetuned			  ERICA--exported.tar.gz.tar.gz
distilroberta-finetuned__exported	  hopperbot-medium
distilroberta-finetuned__exported.tar.gz  runs
ERICA


In [3]:
MODEL_NAME = "../../models/ERICA"

In [4]:
from pathlib import Path

Path(MODEL_NAME).exists()

True

In [17]:
import argparse
import re

import torch
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Conversation,
    ConversationalPipeline,
)

from src.chat import *
from src.classifier import Classifier
from src.utils import DATA_DIR, ROOT_DIR

In [13]:
print(DATA_DIR)
print(ROOT_DIR)

/home/jovyan/work/research/data
/home/jovyan/work/research


In [14]:
CLASSIFIER = ROOT_DIR.parent.resolve().joinpath("models").joinpath("distilroberta-finetuned")

In [15]:
Path(CLASSIFIER).exists()

True

In [97]:
import json

config = AutoConfig.from_pretrained(MODEL_NAME)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    from_tf=False,
    config=config,
)
classifier = Classifier(model=str(CLASSIFIER))

In [98]:
model.config

GPT2Config {
  "_name_or_path": "../../models/ERICA",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_length": 1000,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "no_repeat_ngram_size": 3,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "temperature": 0.8,
  "top_k": 100,
  "top_p": 0.7,
  "torch_dtype": "float32",
  "transformers_versi

In [94]:
config.pad_token_id = tokenizer.eos_token_id
config.max_length = 1000
config.no_repeat_ngram_size = 3
config.do_sample = True
config.top_k = 100
config.top_p = 0.7
config.temperature = 0.8

In [99]:
config.save_pretrained("../../models/ERICA/")

In [100]:
config

GPT2Config {
  "_name_or_path": "../../models/ERICA",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_length": 1000,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "no_repeat_ngram_size": 3,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "temperature": 0.8,
  "top_k": 100,
  "top_p": 0.7,
  "torch_dtype": "float32",
  "transformers_versi

In [20]:
chat(model, tokenizer, classifier=classifier, device="cpu")

>>  I'm feeling depressed


User: I'm feeling depressed
Bot:  Why?


>>  My dad passed away


User: My dad passed away
Bot:  I'm so sorry to hear that. I lost my father recently as well. May he rest in peace.


>>  Thank you. I am sorry for your loss as well.


...

>>  I am proud of building this chatbot.


User: I am proud of building this chatbot.
Bot:  That is great, what did you learn from it so far?


>>  I learned about training bots in an effective way.


User: I learned about training bots in an effective way.
Bot:  What was the method?


KeyboardInterrupt: Interrupted by user

In [23]:
chat_pipeline(model, tokenizer, classifier=classifier, device="cpu")

>>  I'm feeling depressed


User: I'm feeling depressed
Bot:  I'm sorry to hear that. What's wrong?


>>  My dad passed away yesterday


User: My dad passed away yesterday
Bot:  I'm so sorry to hear that. I hope you are doing okay.


>>  Thanks. I am trying my best.


...

>>  I am trying to be positive.


User: I am trying to be positive.
Bot:  That's a good attitude to have.


>>  It really is. I have a more optimistic outlook on life.


User: It really is. I have a more optimistic outlook on life.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1084405/3896992558.py", line 1, in <cell line: 1>
    chat_pipeline(model, tokenizer, classifier=classifier, device="cpu")
  File "/home/jovyan/work/research/src/chat.py", line 82, in chat_pipeline
    result = pipe(conversation)
  File "/opt/conda/lib/python3.10/site-packages/transformers/pipelines/conversational.py", line 239, in __call__
    outputs = super().__call__(conversations, num_workers=num_workers, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py", line 1026, in __call__
    return self.run_single(inputs, preprocess_params, forward_params, postprocess_params)
  File "/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py", line 1033, in run_single
    model_outputs = self.forward(model_inputs, **forward_p

In [24]:
conversation = Conversation()

In [25]:
conversation.add_user_input("I am depressed")

In [29]:
pipe = ConversationalPipeline(
    model=model,
    tokenizer=tokenizer,
    device=-1,
)

pipe.model.config.pad_token_id = pipe.tokenizer.eos_token_id
pipe.model.config.max_length = 1000
pipe.model.config.no_repeat_ngram_size = 3
pipe.model.config.do_sample = True
pipe.model.config.top_k = 100
pipe.model.config.top_p = 0.7
pipe.model.config.temperature = 0.8

In [31]:
pipe.model.config

GPT2Config {
  "_name_or_path": "../../models/ERICA",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_length": 1000,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "no_repeat_ngram_size": 3,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "temperature": 0.8,
  "top_k": 100,
  "top_p": 0.7,
  "torch_dtype": "float32",
  "transformers_versi

In [35]:
conversation = Conversation()
conversation.add_user_input("I'm feeling very depressed")

In [36]:
pipe(conversation)

Conversation id: 83afa87d-d3f0-44af-afc1-aa1f0072bf30 
user >> I'm feeling very depressed 
bot >> I'm sorry to hear that.  Have you thought about seeking professional help? 

In [37]:
conversation.add_user_input("I have tried, but it hasn't been helping")
pipe(conversation)

Conversation id: 83afa87d-d3f0-44af-afc1-aa1f0072bf30 
user >> I'm feeling very depressed 
bot >> I'm sorry to hear that.  Have you thought about seeking professional help? 
user >> I have tried, but it hasn't been helping 
bot >> I know what you mean.  Maybe you should try going to a spiritual service? 

In [38]:
conversation.add_user_input("I don't really believe in that")
pipe(conversation)

Conversation id: 83afa87d-d3f0-44af-afc1-aa1f0072bf30 
user >> I'm feeling very depressed 
bot >> I'm sorry to hear that.  Have you thought about seeking professional help? 
user >> I have tried, but it hasn't been helping 
bot >> I know what you mean.  Maybe you should try going to a spiritual service? 
user >> I don't really believe in that 
bot >> I understand.  Try going to find a local church that will pray for you. 

In [39]:
conversation.add_user_input("I don't go to church")
pipe(conversation)

Conversation id: 83afa87d-d3f0-44af-afc1-aa1f0072bf30 
user >> I'm feeling very depressed 
bot >> I'm sorry to hear that.  Have you thought about seeking professional help? 
user >> I have tried, but it hasn't been helping 
bot >> I know what you mean.  Maybe you should try going to a spiritual service? 
user >> I don't really believe in that 
bot >> I understand.  Try going to find a local church that will pray for you. 
user >> I don't go to church 
bot >> You should try it out.  It will change your life. 

In [40]:
conversation.add_user_input("Religion hasn't been helpful")
pipe(conversation)

Conversation id: 83afa87d-d3f0-44af-afc1-aa1f0072bf30 
user >> I'm feeling very depressed 
bot >> I'm sorry to hear that.  Have you thought about seeking professional help? 
user >> I have tried, but it hasn't been helping 
bot >> I know what you mean.  Maybe you should try going to a spiritual service? 
user >> I don't really believe in that 
bot >> I understand.  Try going to find a local church that will pray for you. 
user >> I don't go to church 
bot >> You should try it out.  It will change your life. 
user >> Religion hasn't been helpful 
bot >> !!!?!!.!!...!!..!! 

In [75]:
conversation = Conversation()


def tokenizer_lengths(conversation):
    return list(map(lambda x: len(tokenizer.encode(x)), conversation.past_user_inputs + conversation.generated_responses))


def chat_bot(text):
    conversation.add_user_input(text)
    print(pipe(conversation))
    token_lengths = tokenizer_lengths(conversation)
    return token_lengths

In [76]:
chat_bot("I can't go to sleep")

Conversation id: 2092a85d-ceeb-464c-b665-586ea999064f 
user >> I can't go to sleep 
bot >> I'm sorry to hear that. What's wrong? 



[6, 11]

In [77]:
chat_bot("I'm worried about the state of the world.")

Conversation id: 2092a85d-ceeb-464c-b665-586ea999064f 
user >> I can't go to sleep 
bot >> I'm sorry to hear that. What's wrong? 
user >> I'm worried about the state of the world. 
bot >> I think that the world is headed in a very positive direction. 



[6, 10, 11, 13]

In [78]:
chat_bot("I disagree, I think we're headed towards a very negative path.")

Conversation id: 2092a85d-ceeb-464c-b665-586ea999064f 
user >> I can't go to sleep 
bot >> I'm sorry to hear that. What's wrong? 
user >> I'm worried about the state of the world. 
bot >> I think that the world is headed in a very positive direction. 
user >> I disagree, I think we're headed towards a very negative path. 
bot >> You're entitled to your opinion_comma_ but I think there's a lot of truth to it. 



[6, 10, 14, 11, 13, 22]

In [79]:
chat_bot("I guess so")

Conversation id: 2092a85d-ceeb-464c-b665-586ea999064f 
user >> I can't go to sleep 
bot >> I'm sorry to hear that. What's wrong? 
user >> I'm worried about the state of the world. 
bot >> I think that the world is headed in a very positive direction. 
user >> I disagree, I think we're headed towards a very negative path. 
bot >> You're entitled to your opinion_comma_ but I think there's a lot of truth to it. 
user >> I guess so 
bot >> So you think Trump is a bad president? 



[6, 10, 14, 3, 11, 13, 22, 9]

In [82]:
chat_bot("Trump was a terrible president")

Conversation id: 2092a85d-ceeb-464c-b665-586ea999064f 
user >> I can't go to sleep 
bot >> I'm sorry to hear that. What's wrong? 
user >> I'm worried about the state of the world. 
bot >> I think that the world is headed in a very positive direction. 
user >> I disagree, I think we're headed towards a very negative path. 
bot >> You're entitled to your opinion_comma_ but I think there's a lot of truth to it. 
user >> I guess so 
bot >> So you think Trump is a bad president? 
user >> Trump was a terrible president 
bot >> !!!?!!.!!...!!..!! 



[6, 10, 14, 3, 5, 11, 13, 22, 9, 9]

In [83]:
sum([6, 10, 14, 3, 5, 11, 13, 22, 9, 9])

102